In [2]:
pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/4a/b7/134389cfd027b5c65758b5bd8c2ff06965e9cac8c4fba867e32fae8ddf32/tensorflow-2.16.1-cp310-cp310-macosx_10_15_x86_64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/packages/fa/39/5aae5

In [2]:
pip install mediapipe

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/b8/d3/cd033dcbe630e8dc8b6e5658ad45383d978f4069dd62857e900677f3f1ee/mediapipe-0.10.11-cp310-cp310-macosx_11_0_x86_64.whl.metadata
  Obtaining dependency information for jax from https://files.pythonhosted.org/packages/dc/d9/f387d9dfb2cf00f814b24e0f8bf6f4c68ae01870994dc436993fadd73563/jax-0.4.26-py3-none-any.whl.metadata
  Obtaining dependency information for jaxlib from https://files.pythonhosted.org/packages/19/1c/f59eda5f4a382720fe986268b8c148ab67b74eef28b3e20536ffce99462a/jaxlib-0.4.26-cp310-cp310-macosx_10_14_x86_64.whl.metadata
  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/16/07/bf25df600eeaedddf8fece3f1ff837bf72865b93a03651cf7375ce8172be/opencv_contrib_python-4.9.0.80-cp37-abi3-macosx_10_16_x86_64.whl.metadata
  Obtaining dependency information for protobuf<4,>=3.11 from https://files.pythonhosted.org/packages/8d/14/619e24a4c70df

In [13]:
pip install mediapipe

/Users/mac/miniforge3/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --upgrade mediapipe

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, Input, optimizers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import pad_sequences
import mediapipe as mp
import cv2
import uuid
import os
import json

2024-04-09 12:44:12.059219: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
lip_marks = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 78, 191, 80, 81, 82, 13, 312,
             311, 310, 415, 308, 95, 88, 178, 87, 14, 317, 402, 318, 324, 146, 91, 181, 84,
             17, 314, 405, 321, 375]

In [3]:
# Utilities

def extract_coordinates(results):
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]) if results.face_landmarks else np.zeros((468, 3))
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]) if results.pose_landmarks else np.zeros((33, 3))
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]) if results.left_hand_landmarks else np.zeros((21, 3))
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]) if results.right_hand_landmarks else np.zeros((21, 3))
    return np.concatenate([face, lh, pose, rh])

def draw(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(0,0,255), thickness=3, circle_radius=3),
                              mp_drawing.DrawingSpec(color=(0,0,0), thickness=1, circle_radius=0))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(0,150,0), thickness=3, circle_radius=3),
                              mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(200,56,12), thickness=3, circle_radius=3),
                              mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(250,56,12), thickness=3, circle_radius=3),
                              mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2))

In [4]:
# Helper Function

def load_json_file(json_path):
    with open(json_path, 'r') as f:
        sign_map = json.load(f)
    return sign_map

class CFG:
    data_dir = "asl-signs/"
    sequence_length = 12
    rows_per_frame = 543

ROWS_PER_FRAME = 543
sequence = []
def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    print('pq_path len: ', len(pq_path))
    sequence.append(pq_path)
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)
    
sign_map = load_json_file(CFG.data_dir + 'sign_to_prediction_index_map.json')
train_data = pd.read_csv(CFG.data_dir + 'train.csv')

s2p_map = {k.lower():v for k,v in load_json_file(CFG.data_dir + "sign_to_prediction_index_map.json").items()}
p2s_map = {v:k for k,v in load_json_file(CFG.data_dir + "sign_to_prediction_index_map.json").items()}
encoder = lambda x: s2p_map.get(x.lower())
decoder = lambda x: p2s_map.get(x)

# Processing the video

In [5]:
def predict_asl(mode=0, video_path='', model=''):
    
    if model=='ann':
        path='models/ann/model.tflite'
    elif model=='top-01':
        path='models/1st-place-solution-by-hoyso48/model.tflite'
    elif model=='cnn':
        path='models/cnn/model.tflite'
    elif model=='cnn+3trans':
        path='models/cnn+3trans/model.tflite'
    elif model=='lstm':
        path='models/lstm/model.tflite'
    elif model=='transformer':
        path='models/transformer/model.tflite'
    else:
        path='models/distance-angle-based-features-using-keras/model.tflite'
        
    interpreter = tf.lite.Interpreter(model_path=path)
    interpreter.allocate_tensors()
    print('Initilized Tensors')
    found_signatures = list(interpreter.get_signature_list().keys())
    prediction_fn = interpreter.get_signature_runner("serving_default")
    
    coordinates = []
    res = []
    
    cap = cv2.VideoCapture(0) if mode==0 else cv2.VideoCapture(video_path)
        
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while not cap.isOpened():
            print('Capture is Not Open')
            cap = cv2.VideoCapture(0) if mode==0 else cv2.VideoCapture(video_path)
            cv2.waitKey(1000)

        pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        print('Starting Prediction')
        while True:
            flag, image = cap.read()
            if flag:
                image.flags.writeable = False
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = holistic.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                landmarks = extract_coordinates(results)
#                 draw(image, results)
                coordinates.append(landmarks)
                if len(coordinates) == 15:
                    prediction = prediction_fn(inputs=np.array(coordinates).astype(np.float32))
                    sign = np.argmax(prediction["outputs"])
#                     print(f'{decoder(sign)} {format(prediction["outputs"][sign]*10,".2f")}%')
                    if model=='cnn+3trans' or model=='lstm':
                        if prediction["outputs"][0][sign]*10 > 50:
                            res.append(decoder(sign))
                    elif prediction["outputs"][sign]*10 > 50:
                            res.append(decoder(sign))
                    cv2.putText(image, f"Prediction:    {decoder(sign)}", (3, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    coordinates = coordinates[10:]
                    
                
                cv2.imshow('Prediction',image)

            else:
                # The next frame is not ready, so we try to read it again
                cap.set(cv2.CAP_PROP_POS_FRAMES, pos_frame-1)
                # It is better to wait for a while for the next frame to be ready
                cv2.waitKey(1000)
                break

            if cv2.waitKey(10) & 0xFF == 27:
                break
            if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
                # If the number of captured frames is equal to the total number of frames,
                # we stop
                break
    print(max(res, key=res.count) if len(res)>0 else "Couldn't Predict")
    cap.release()
    cv2.destroyAllWindows()

In [6]:
predict_asl(1, 'datasets/downloaded/animal/animal.mp4', model='cnn+3trans')

Initilized Tensors


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


RuntimeError: ValidatedGraphConfig Initialization failed.
ImageToTensorCalculator: ; RET_CHECK failure (mediapipe/calculators/tensor/image_to_tensor_calculator.cc:144) ValidateOptionOutputDims(options) returned INTERNAL: ; RET_CHECK failure (./mediapipe/calculators/tensor/image_to_tensor_utils.h:136) options.has_output_tensor_float_range() || options.has_output_tensor_int_range() || options.has_output_tensor_uint_range()Output tensor range is required. 
InferenceCalculatorCpu: ; RET_CHECK failure (mediapipe/calculators/tensor/inference_calculator_cpu.cc:63) !options.model_path().empty() ^ kSideInModel(cc).IsConnected()Either model as side packet or model path in options is required.
SplitDetectionVectorCalculator: The number of output streams should match the number of ranges specified in the CalculatorOptions.
ImageToTensorCalculator: ; RET_CHECK failure (mediapipe/calculators/tensor/image_to_tensor_calculator.cc:144) ValidateOptionOutputDims(options) returned INTERNAL: ; RET_CHECK fa

In [7]:
predict_asl(model='cnn+3trans')

Initilized Tensors


RuntimeError: ValidatedGraphConfig Initialization failed.
ImageToTensorCalculator: ; RET_CHECK failure (mediapipe/calculators/tensor/image_to_tensor_calculator.cc:144) ValidateOptionOutputDims(options) returned INTERNAL: ; RET_CHECK failure (./mediapipe/calculators/tensor/image_to_tensor_utils.h:136) options.has_output_tensor_float_range() || options.has_output_tensor_int_range() || options.has_output_tensor_uint_range()Output tensor range is required. 
InferenceCalculatorCpu: ; RET_CHECK failure (mediapipe/calculators/tensor/inference_calculator_cpu.cc:63) !options.model_path().empty() ^ kSideInModel(cc).IsConnected()Either model as side packet or model path in options is required.
SplitDetectionVectorCalculator: The number of output streams should match the number of ranges specified in the CalculatorOptions.
ImageToTensorCalculator: ; RET_CHECK failure (mediapipe/calculators/tensor/image_to_tensor_calculator.cc:144) ValidateOptionOutputDims(options) returned INTERNAL: ; RET_CHECK fa